# Ilastik-like napari

## Import packages

In [1]:
import os 
import warnings
warnings.filterwarnings('ignore')
import napari
import skimage
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import skimage.filters
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


## Create the napari view

In [2]:
%gui qt5

We work with the skimage coins image

In [3]:
image = skimage.data.coins()

The view has three layers: image_layer for the image, labels_layer to draw labels, and seg_layer to show the prediction:

In [4]:
view = napari.Viewer()
#image layer
image_layer = view.add_image(image)

#labels layer. We create two labelled single-pixels for the two categories, so that 
#sklearn stays happy after one draws the first label
labels_layer = view.add_labels(data = np.zeros(image.shape),name = 'label1')
labels_layer.data[0,0]=1
labels_layer.data[1,1]=2

#prediction layer
seg_layer = view.add_image(data = np.zeros(image.shape),name = 'predict',
                           opacity = 0.2, contrast_limits = [0,2], colormap = 'hsv')

## Series of filters for ML

We create a series of filtered images to capture different types of information from the image. If one uses N filters, one has N features per pixel to learn.

In [5]:
im_gauss = skimage.filters.gaussian(image,sigma = 10, preserve_range = True)
im_gauss2 = skimage.filters.gaussian(image,sigma = 20, preserve_range = True)
im_frangi = skimage.filters.frangi(image)
im_prewitt = skimage.filters.prewitt(image)
im_meijering = skimage.filters.meijering(image)
im_gauss_laplace = skimage.filters.laplace(skimage.filters.gaussian(image,sigma = 5,preserve_range=True),ksize=10)
im_gradient = skimage.filters.rank.gradient(image,skimage.morphology.disk(8))
im_entropy = skimage.filters.rank.entropy(image,skimage.morphology.disk(8))
im_roberts = skimage.filters.roberts(skimage.filters.gaussian(image,sigma = 5,preserve_range=True))

all_filt = [im_gauss,im_gauss2,im_frangi,im_prewitt,im_meijering,im_gauss_laplace,im_gradient,im_entropy,im_roberts]
names = ['Gauss $\sigma$=10', 'Gauss $\sigma$=20','Frangi','Prewitt','Meijering','Gauss+Laplace', 'Gradient',
        'Entropy','Roberts']

Here we add the series of filtered images to the napari viewer. There's probably a more elegant way to do that.

In [6]:
view.all_filt = all_filt
view.names = names

## Interactive view

Finally we use a mouse callback to update the ML calculated segmentation after each addition of a drawn region.

In [7]:
@view.mouse_drag_callbacks.append
def mouse_callback(viewer, event):
    
    #clicking
    yield
    #dragging
    while event.type == 'mouse_move':
        yield
    
    #create feature and targets dataframes/series
    features = pd.DataFrame(index=np.arange(np.sum(viewer.layers[1].data>0)))
    
    for ind, x in enumerate(viewer.names):
        features[x] = np.ravel(viewer.all_filt[ind][viewer.layers[1].data>0])

    targets = pd.Series(np.ravel(viewer.layers[1].data[viewer.layers[1].data>0]).astype(int))

    #split train/test
    X, X_test, y, y_test = train_test_split(features, targets, 
                                        test_size = 0.2, 
                                        random_state = 42)
    
    #train a random forest classififer
    random_forrest = RandomForestClassifier(n_estimators=1000)
    random_forrest.fit(X, y)
    
    #classify all pixels and update the segmentation layer
    all_pixels = pd.DataFrame(index=np.arange(len(np.ravel(viewer.layers[0].data))))
    for ind, x in enumerate(viewer.names):
        all_pixels[x] = np.ravel(viewer.all_filt[ind])
    predictions = random_forrest.predict(all_pixels)
    
    predicted_image = np.reshape(predictions, viewer.layers[0].data.shape)
    
    viewer.layers[2].data = predicted_image.astype(np.uint8)
    